# Recap de la manip kaggle (general contest):


* analyse du sujet
* recuperer le train data set et le test data set (a submitter à kaggle)
* lecture des features et reperer la target 

* On importe les librairies

In [2]:
from pandas import DataFrame, read_csv
import bigml.api
from bigml.api import BigML
from sklearn.model_selection import train_test_split



* Importer les data set CSV 

In [ ]:
!pwd
!ls
fulltrain=read_csv('./cs-training.csv', index_col=0)
test=read_csv('./cs-test.csv', index_col=0)
fulltrain.head(2)

* Realisation d'un split de 80/20 sur le train full de kaggle afin d'entrainer un modele
* On a donc un Train set (80%) et un Dev set(20%)

In [44]:
training_set, dev_set = train_dev_split(fulltrain, test_size=0.2)

export_csv = dev_set.to_csv (r'dev_set.csv',  header=True) 
export_csv = training_set.to_csv (r'training_set.csv',  header=True)

## Création du modele
* Sur BigML
  * On créer une source
  * On créer un data set depuit la source 
  * On créer notre modèle (ici ensemble)

In [50]:
source = api.create_source('training_set.csv')
# waiting for the source to be finished. Results will be stored in `source`
api.ok(source)
# step 3: creating a dataset from the previously created `source`
dataset = api.create_dataset(source)
# waiting for the dataset to be finished
api.ok(dataset) 
# step 5: creating an Ensemble
ensemble1 = api.create_ensemble(dataset)
# waiting for the ensemble to be finished
api.ok(ensemble)

True

##  On fait une évaluation du modèle

* on regarde les summury report pour voir quels features on été le plus utilisés par le modele.


In [ ]:
importances = ensemble['object']['importance']

* refabriquer un dictionnaire avec les noms de colonnes (et non plus les n°)

In [ ]:

importances_named = dict()
for column, importance in importances.items():
    column_name = ensemble['object']['ensemble']['fields'][column]['name']
    importances_named[column_name] = importance

* on fait une batchprediction de notre modèle sur le dev set

In [ ]:
# step 6: creating a source from the data in your local "data/test_iris.csv" file
dev_source = api.create_source("dev_set.csv")
# waiting for the source to be finished. Results will be stored in `source`
api.ok(dev_source)
# step 8: creating a dataset from the previously created `source`
dev_dataset = api.create_dataset(dev_source)
# waiting for the dataset to be finished
api.ok(dev_dataset)
# step 10: creating a batch prediction
batch_prediction = api.create_batch_prediction(ensemble, dev_dataset,
                                               {"all_fields": True})
# waiting for the batch_prediction to be finished
api.ok(batch_prediction)
# downloading the results to your computer
api.download_batch_prediction(batch_prediction,
                              filename='predictions/my_predictions.csv')

## on créer l'evaluation
* On regarde les matrices de confusions, l AUC, Accuracy... afin de comprendre la precision du modele.
      

test_dataset = api.create_dataset(test_source)
api.ok(test_dataset)
evaluation = api.create_evaluation(ensemble, test_dataset)
api.ok(evaluation)
evaluation

* on adapte nos data set en ajoutant des features, ponderant, etc... en se basant sur le summary report precedent.
  * On peut egalement changer de modele
  * on peut faire varier les splits ratio

## Lorsque que l'on est satisfait de notre modele de prediction

* On peut entrainer notre modele sur le train full pour augmenter sa capacité à généraliser.

In [51]:
source = api.create_source('training_full.csv')
# waiting for the source to be finished. Results will be stored in `source`
api.ok(source)
# step 3: creating a dataset from the previously created `source`
dataset = api.create_dataset(source)
# waiting for the dataset to be finished
api.ok(dataset) 
# step 5: creating an Ensemble
ensemble = api.create_ensemble(dataset)
# waiting for the ensemble to be finished
api.ok(ensemble)

True

* On applique le modele obtenu  sur le test (fichier kaggle)
* on enregristre en csv le resultat test full avec la colonne de prediction

In [74]:

# step 6: creating a source from the data in your local "data/test_iris.csv" file
test_source = api.create_source("test_set.csv")
# waiting for the source to be finished. Results will be stored in `source`
api.ok(test_source)
# step 8: creating a dataset from the previously created `source`
test_dataset = api.create_dataset(test_source)
# waiting for the dataset to be finished
api.ok(test_dataset)
# step 10: creating a batch prediction
batch_prediction = api.create_batch_prediction(ensemblefull, validation_dataset,
                                               {"all_fields": True,
                                                "header": True,
                                                "confidence": True,
                                                "probabilities": True})
# waiting for the batch_prediction to be finished
api.ok(batch_prediction)
# downloading the results to your computer
api.download_batch_prediction(batch_prediction,
                              filename='predictions/my_predictions_test.csv')

'predictions/my_predictions3.csv'

* on mets en forme le fichier selon la demande de kaggle

In [7]:
prediction=read_csv('predictions/my_predictions_test.csv',index_col=False)
print(list(prediction))

['field1', 'RevolvingUtilizationOfUnsecuredLines', 'age', 'DebtRatio', 'MonthlyIncome', 'MonthlyDebt', 'DisposableIncome', 'BalancedIncome', 'NumberOfOpenCreditLinesAndLoans', 'NumberRealEstateLoansOrLines', 'NumberOfTime30-59DaysPastDueNotWorse', 'NumberOfTime60-89DaysPastDueNotWorse', 'NumberOfTimes90DaysLate', 'WeightedOfLatePayment', 'NumberOfDependents', 'SeriousDlqin2yrs', 'SeriousDlqin2yrs.1', 'confidence', '0 probability', '1 probability']


In [ ]:
kaggle_prediction=DataFrame()
kaggle_prediction['Id']=prediction['field1']
kaggle_prediction['Probability']=prediction['1 probability']

In [9]:
kaggle_prediction.head()

,Id,Probability
0,1,0.05203
1,2,0.03826
2,3,0.00986
3,4,0.09709
4,5,0.13328


* on upload le csv sur kaggle pour decouvrir le score de prediction et le leader board.

In [10]:
kaggle_prediction_file="kaggleprediction.csv"
kaggle_prediction.to_csv(kaggle_prediction_file,index=False)
kaggle.api.competition_submit(kaggle_prediction_file, "Prediction Bigml", "GiveMeSomeCredit")

100%|██████████| 1.65M/1.65M [00:05<00:00, 333kB/s] 


Successfully submitted to Give Me Some Credit

Maud, Maxime, Laurent, Christophe